# Model

Energy Systems Model based on the Open Energy Modelling Framework (oemof). 

**Select a scenario:**

In [ ]:
scenario = "scenario-A"  # name of the scenario and the corresponding file, e.g. "scenario-A.xls"

In [ ]:
import os
import pkg_resources as pkg
import pandas as pd

from pyomo.opt import SolverFactory
from oemof.solph import EnergySystem, Model, Bus
from oemof.tools.economics import annuity as annuity
from oemof.solph import constraints
import oemof.tabular.tools.postprocessing as pp
import oemof.tabular.facades as fc

## Creating and Setting the Datapaths

First, the datapath for raw-data and results is set. Data handling looks more complex than it is. You can easily adapt this to a simple `pd.read_excel(filepath,...)` in the next block if your file is located somewhere else. Otherwise we will use data from the repository repository. 

In addition a results directory will be created in `home/user/oemof-barbados/results/<scenario-name>/output`. 

In [ ]:
# datapath for input data from the oemof tabular pacakge
datapath = os.path.join(
    os.getcwd(),  # pkg.resource_filename("oemof.jordan", ""),
    "data",
    scenario + ".xls",
)

carrier_technology_path = os.path.join(os.getcwd(), "data", "carrier-technology.xls")

# results path
results_path = os.path.join(os.path.expanduser("~"), "oemof-barbados", "results")

scenario_path = os.path.join(results_path, scenario, "output")

if not os.path.exists(scenario_path):
    os.makedirs(scenario_path)
print("Results will be stored in {}".format(scenario_path))

Next required input data will be read. The profiles index will be used for the `EnergySystem` object below. 
All generator data etc. will also be loaded. 

In [ ]:
file = pd.ExcelFile(datapath)

sheet_names = [typ for typ in file.sheet_names if typ in fc.TYPEMAP.keys()]

data = {}

for sheet in sheet_names:
    data[sheet] = pd.read_excel(datapath, sheet_name=sheet, index_col=0)

# profiles and tech data not be part of TYPEMAP and need to be read seperately
profiles = pd.read_excel(
    datapath, sheet_name="profiles", index_col=[0], parse_dates=True,
)
profiles.index.freq = "1H"

technology = pd.read_excel(
    carrier_technology_path, sheet_name="technology-data", index_col=[0, 1, 2]
)
carrier = pd.read_excel(carrier_technology_path, sheet_name="carrier", index_col=[0, 1])

In [ ]:
all_components = pd.concat([v for k, v in data.items() if k != "bus"], sort=False)
# Only be used for Latex export of tables
# columns = ['profile', 'capacity_potential']
# print(all_components.to_latex(columns=columns, na_rep="-"))

In [ ]:
def calc_annuity(u, scenario):
    capacity_cost = (
        annuity(
            technology.at[(u.carrier, u.tech, scenario), "capex"],
            technology.at[(u.carrier, u.tech, scenario), "lifetime"],
            technology.at[(u.carrier, u.tech, scenario), "wacc"],
        )
        + technology.at[(u.carrier, u.tech, scenario), "fom"]
    )
    return capacity_cost

## Creating the EnergySystem and its Nodes

First a `EnergySystem` object is created which will hold all information (nodes, etc.) of hour energy system that are added below. This is just the standard way of using the `oemof.solph` library for modelling.

In [ ]:
es = EnergySystem(timeindex=profiles.index)

### Add Bus

Before any component is added all bus objects for the model are created and added to the energy system object.

In [ ]:
buses = {
    name: Bus(label=name, balanced=bool(arg.balanced))
    for name, arg in data["bus"].iterrows()
}
es.add(*buses.values())


#### Bus Constraints 

With the set of all Buses $B$ all inputs $x^{flow}_{i(b),b}$ to a bus $b$ must equal all its outputs $x^{flow}_{b,o(b)}$

$$\sum_i x^{flow}_{i(b), b}(t) - \sum_o x^{flow}_{b, o(b)}(t) = 0 \qquad \forall t \in T, \forall b \in B$$

This equation will be build once the complete energy system is setup with its component. Every time a `Component` is created, the connected bus inputs/outputs will be updated. By this update every bus has all required information of its inputs and outputs available to construct the constraints. 


### Add Load


In [ ]:
for name, l in data["load"].iterrows():
    es.add(
        fc.Load(
            label=name,
            bus=buses[
                l.bus
            ],  # reference the bus in the buses dictionary
            amount=l.amount,  # amount column
            profile=profiles[l.profile],
        )
    )


#### Load Constraint 

For the set of all Load denoted with $l \in L$ the load $x_l$ at timestep t equals the exogenously defined  profile value $c^{profile}_l$ multiplied by the amount of this load $c^{amount}_l$.

$$ x^{flow}_{l}(t) = c^{profile}_{l}(t) \cdot c^{amount}_{l} \qquad \forall t \in T, \forall l \in L$$

### Add Generators


In [ ]:
for name, g in data["dispatchable"].iterrows():
    # calculates marginal costs from carrier cost and efficiency if not set 
    if "marginal_cost" not in g.index: 
        marginal_cost=(
                carrier.at[(g.carrier, scenario), "cost"]
                / technology.at[
                    (g.carrier, g.tech, scenario), "efficiency"
                ]
            ) + technology.at[
                    (g.carrier, g.tech, scenario), "vom"
                ]
    else:
        # otherwise use specified marginal cost from sheet 
        marginal_cost = g.marginal_cost
        
    es.add(
        fc.Dispatchable(
            label=name,
            bus=buses[g.bus],
            carrier=g.carrier,
            tech=g.tech,
            marginal_cost = marginal_cost,
            efficiency=technology.at[(g.carrier, g.tech), 'efficiency'],
            expandable=bool(g.expandable),
            capacity=g.capacity,
            capacity_potential=g.capacity_potential,
            capacity_cost=calc_annuity(g, scenario) * 1000,  # to $/MW
            output_parameters={
                "emission_factor": (
                    carrier.at[(g.carrier, scenario), "emission_factor"]
                    / technology.at[
                        (g.carrier, g.tech, scenario), "efficiency"
                    ]
                )
            },
        )
    )


#### Dispatchable Generator Constraint

A `Generator` component can be used to model all types of dispatchble units in a energy system. This can include diesel generators oder coal fired power plants but also hot water boilers for heat. Every generator **must** be connected to an `Bus` object. 

This basic mathematical model for the component with the set of all dispatchable generators being $d \in D$ looks as follows:

$$x^{flow}_{d}(t) \leq c^{capacity}_{d} \qquad \forall t \in T,  \forall d \in D$$

Meaning, the production of the generator $x^{flow}$ must be less than its maximum capacity $c^{capacity}$ in every timestep. If the `exapandable` attribute is set, the flow variable will be bounded by a variable $x^{capacity}_d$ which is subject to optimisation instead of the parameter. 

In [ ]:
for name, v in data["volatile"].iterrows():
    es.add(
        fc.Volatile(
            label = name,
            bus = buses[v.bus],
            carrier = v.carrier,
            tech = v.tech,
            expandable = bool(v.expandable),
            capacity = v.capacity,
            capacity_potential = v.capacity_potential,
            capacity_cost = calc_annuity(v, scenario) * 1000,  # $/kW -> $/MW
            profile = profiles[v.profile],
        )
    )


#### Volatile Generator Constraint 

Using the `Generator` component with `output_parameters={"fixed": True}` is very similar to the Dispatchable component. However, in this case the flow of the volatile components denoted with $v \in V$  will be fixed to a specific value.

$$ x^{flow}_{v}(t) = c^{profile}_{v}(t) \cdot x^{capacity}_{v} \qquad \forall t \in T, \forall v \in V$$


### Add Storage

In [ ]:
for name, s in data["storage"].iterrows():
    es.add(
        fc.Storage(
            label=name,
            bus=buses[s.bus],
            carrier=s.carrier,
            tech=s.tech,
            marginal_cost=s.marginal_cost,
            capacity=s.capacity,
            storage_capacity=s.storage_capacity,
            storage_capacity_potential=s.storage_capacity_potential,
            expandable=bool(s.expandable),
            initial_storage_level=s.initial_storage_level,
            efficiency=technology.at[
                (s.carrier, s.tech, scenario), "efficiency"
            ],
            loss_rate=s.loss_rate,
            storage_capacity_cost=annuity(
                technology.at[
                    (s.carrier, s.tech, scenario), "storage_capex"
                ],
                technology.at[
                    (s.carrier, s.tech, scenario), "lifetime"
                ],
                 technology.at[
                    (s.carrier, s.tech, scenario), "wacc"
                ],
            ) * 1000,  # $/kW -> $/MW
            capacity_cost=calc_annuity(s, scenario) * 1000,  # $/kW -> $/MW
        )
    )


#### Storage Constraints 

The mathematical representation of the storage for all storages $s \in S$ will include the flow into the storage, out of the storage and a storage level. The defaul efficiency for input/output is 1. Note that this is is included during charge and discharge. If you want to set the round trip efficiency you need to do for example: $\eta = \sqrt{\eta^{roundtrip}}$

Intertemporal energy balance of the storage:

$$ x^{level}_{s}(t) = (1-\eta^{loss}) x^{level}_{s}(t) + \eta_{in} x^{flow}_{s, in} - \frac{x^{flow}_{s, out}(t)}{\eta_{out}} \qquad \forall t \in T,  \forall s \in S$$ 

Bounds of the storage level variable $x^{level}_s(t)$:

$$ x^{level}_s(t) \leq c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$


$$ x^{level}_s(1) = x_s^{level}(t_{e}) = 0.5 \cdot c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$ 

Of course, in addition the inflow/outflow of the storage also needs to be within the limit of the minimum and maximum power. 

$$ -c_s^{capacity} \leq x^{flow}_s(t) \leq c_s^{capacity} \qquad \forall t \in T, \forall s \in S$$ 


### Add Conversion 

A conversion unit will take from a bus and feed into another: 

$$x^{flow}_{c, to}(t) = c^{efficiencty}_{c} \cdot x^{flow}_{c, from}(t), \qquad \forall c  \in C, \forall t \in T$$ 

In [ ]:
for name, c in data["conversion"].iterrows():
    es.add(
        fc.Conversion(
            label=name,
            from_bus=buses[c.from_bus],
            to_bus=buses[c.to_bus],
            carrier=c.carrier,
            tech=c.tech,
            efficiency=technology.at[
                (c.carrier, c.tech, scenario), "efficiency"
            ],
            carrier_cost=carrier.at[(c.carrier, scenario), "cost"],
            expandable=bool(c.expandable),
            capacity=c.capacity,
            capacity_potential=c.capacity_potential,
            capacity_cost=calc_annuity(c, scenario)* 1000,  # $/kW -> $/MW
            output_parameters={
                "emission_factor": (
                    carrier.at[(c.carrier, scenario), "emission_factor"]
                    / technology.at[
                        (c.carrier, c.tech, scenario), "efficiency"
                    ]
                )
            },
        )
    )


### Add Commodity

In [ ]:
for name, c in data["commodity"].iterrows():
    es.add(
        fc.Commodity(
            label=name,
            bus=buses[c.bus],
            carrier=c.carrier,
            tech=c.tech,
            amount=c.amount,
        )
    )


### Add Link

In [ ]:
for name, c in data["link"].iterrows():
    es.add(
        fc.Link(
            label=name,
            from_bus=buses[c.from_bus],
            to_bus=buses[c.to_bus],
            capacity=c.capacity,
            expandable=bool(c.expandable),
            capacity_cost=calc_annuity(l, scenario) * 1000,  # $/kW -> $/MW
            loss=c.loss,
        )
    )

### Objective Function 

The objective function is created from all instantiated objects. It will use all operating costs (i.e. `marginal_cost` argument) and if set all investment costs (i.e. `capacity_cost` argument)

$$ \text{min:} \sum_g \sum_t \overbrace{c^{marginal\_cost}_g \cdot x^{flow}_{g}(t)}^{\text{operating cost}} \\ 
\sum_g \sum_t \overbrace{c^{capacity\_cost}_g \cdot x^{capacity}_{g}(t)}^{\text{investment cost}} $$

### Add Shortage/Excess Slack Components

In [ ]:
for name, e in data["excess"].iterrows():
    es.add(fc.Excess(label=name, bus=buses[e.bus]))

for name, s in data["shortage"].iterrows():
    es.add(
        fc.Shortage(
            label=name,
            carrier="electricity",
            tech="shortage",
            bus=buses[s.bus],
            marginal_cost=s.marginal_cost,
        )
    )


## Creating the Mathematical Model

In [ ]:
# create model based on energy system and its components
m = Model(es)

# inspect objective function
# m.objective.pprint()

m.receive_duals()

### Add CO2 Constraint

To add a CO2-constraint we will use the `oemof.solph.constraints` module which allows to add such a constraint in a easy way. 

$$ \sum_t \sum_f  x^{flow}_f(t) \cdot c^{emission\_factor}_f \leq \overline{L_{CO_2}} $$

The constraint will sum all flows for the complete time horzion that have an attribute `emission_factor` and multiple the flow value with this factor. 

In [ ]:
constraints.emission_limit(m, limit=0)
# m.write(io_options={"symbolic_solver_labels":True})

## Solving the Model and Writing Results

In [ ]:
# check if cbc solver library is available
solver = "gurobi"
solver_available = SolverFactory(solver).available()

if solver:
    #  solve the model using cbc solver
    m.solve(solver, tee=True)

    # write results back to the model object
    m.results = m.results()

    # writing results with the standard oemof-tabular output formatt
    pp.write_results(m, scenario_path)

    print(
        "Optimization done. Results are in {}.".format(
            results_path
        )
    )
